In [1]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
!pip install 'shimmy>=2.0'

In [3]:
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from scipy.sparse import csr_matrix, issparse
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import cosine



In [4]:
# Define RL Environment for Eigenvector Search
class EigenvectorEnv(gym.Env):
    def __init__(self, A, target_eigenvalue):
        super(EigenvectorEnv, self).__init__()
        self.A = A
        self.target_eigenvalue = target_eigenvalue
        self.dim = A.shape[0]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.dim,), dtype=np.float32)
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.dim,), dtype=np.float32)
        self.state = None

    def reset(self):
        self.state = np.random.randn(self.dim)
        self.state /= np.linalg.norm(self.state)
        return self.state

    def step(self, action):
        action = np.array(action)
        action /= np.linalg.norm(action) + 1e-8
        self.state = action
        # Reward is negative of the residual norm: ||Ax - λx||
        if issparse(self.A):
            residual = self.A.dot(action) - self.target_eigenvalue * action
        else:
            residual = self.A @ action - self.target_eigenvalue * action
        reward = -np.linalg.norm(residual)
        done = False
        return self.state, reward, done, {}

# Function to compute cosine distance
def cosine_distance(v1, v2):
    v1 = v1 / (np.linalg.norm(v1) + 1e-8)
    v2 = v2 / (np.linalg.norm(v2) + 1e-8)
    cos_sim = np.abs(np.dot(v1, v2))
    cos_sim = min(cos_sim, 1.0)  # Handle numerical errors
    return np.arccos(cos_sim) / np.pi  # Normalized to [0, 1]

# Function to find dominant eigenvalue and compute eigenvector using RL
def compute_dominant_eigenvector(A, timesteps=20000, eval_steps=100):
    # Ensure matrix is symmetric
    if issparse(A):
        A = (A + A.T) / 2
    else:
        A = (A + A.T) / 2

    # Compute dominant eigenvalue and eigenvector using scipy
    eigvals, eigvecs = eigsh(A, k=1, which='LA') if issparse(A) else np.linalg.eigh(A)
    dominant_eigenvalue = eigvals[-1] if not issparse(A) else eigvals[0]
    true_eigenvector = eigvecs[:, -1] if not issparse(A) else eigvecs[:, 0]
    print("Dominant eigenvalue (scipy):", dominant_eigenvalue)

    # Initialize environment and RL model
    env = EigenvectorEnv(A, dominant_eigenvalue)
    model = PPO('MlpPolicy', env, verbose=1)
    model.learn(total_timesteps=timesteps)

    # Evaluate to find best eigenvector
    obs = env.reset()
    best_reward = -np.inf
    best_vec = None

    for _ in range(eval_steps):
        action, _ = model.predict(obs)
        obs, reward, _, _ = env.step(action)
        if reward > best_reward:
            best_reward = reward
            best_vec = obs

    print("Approximated eigenvector (RL):", best_vec)
    # Compute residual norm for verification
    residual_norm = np.linalg.norm(A @ best_vec - dominant_eigenvalue * best_vec) if not issparse(A) else np.linalg.norm(A.dot(best_vec) - dominant_eigenvalue * best_vec)
    print("Residual norm ||Ax - λx||:", residual_norm)
    # Compute cosine distance for accuracy
    cos_dist = cosine_distance(best_vec, true_eigenvector)
    print("Cosine distance to true eigenvector:", cos_dist)

    return dominant_eigenvalue, best_vec, cos_dist

# Test on dense matrix

In [6]:
# Test with dense symmetric matrix
np.random.seed(42)
size = 5
A_dense = np.random.randn(size, size)
A_dense = (A_dense + A_dense.T) / 2
print("\nTesting with dense matrix:")
eigenvalue, eigenvector, cos_dist_dense = compute_dominant_eigenvector(A_dense)




Testing with dense matrix:
Dominant eigenvalue (scipy): 1.991113406258497
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 846  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 524         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008063999 |
|    clip_fraction        | 0.0597      |
|    clip_range           | 0.2         |
|    entropy_loss         | -7.11       |
|    explained_variance   | -9.89e-06   |
|    learning_rate        | 0.0003      |
|    loss                 | 419         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0119     |
|    std                  | 1           |
|    value_loss           | 1.24e+03    |
----------------------------------

# Test on Large sparse matrix

In [7]:
# Test with large sparse symmetric matrix
from scipy.sparse import random as sparse_random
np.random.seed(42)
size = 1000
A_sparse = sparse_random(size, size, density=0.01, format='csr')
print("\nTesting with sparse matrix:")
eigenvalue, eigenvector, cos_dist_sparse = compute_dominant_eigenvector(A_sparse)


Testing with sparse matrix:
Dominant eigenvalue (scipy): 5.3497582686143055
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 677  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 559         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.072074465 |
|    clip_fraction        | 0.32        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.42e+03   |
|    explained_variance   | -0.00059    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.25e+03    |
|    n_updates            | 10